
# MusicGen
Welcome to MusicGen's demo jupyter notebook. Here you will find a series of self-contained examples of how to use MusicGen in different settings.

First, we start by initializing MusicGen, you can choose a model from the following selection:
1. `facebook/musicgen-small` - 300M transformer decoder.
2. `facebook/musicgen-medium` - 1.5B transformer decoder.
3. `facebook/musicgen-melody` - 1.5B transformer decoder also supporting melody conditioning.
4. `facebook/musicgen-large` - 3.3B transformer decoder.

We will use the `facebook/musicgen-small` variant for the purpose of this demonstration.

In [ ]:
!pip install MusicGen
!pip install audiocraft
!pip install openai==0.28
!pip install langchain



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
config_ini_location = '/content/drive/MyDrive/IS883/config_new1.ini' # Change this to point to the location of your config.ini file.

import configparser

config = configparser.ConfigParser()
config.read(config_ini_location)
openai_api_key = config['OpenAI']['API_KEY']

In [ ]:
csv_path = "/content/drive/MyDrive/IS883 Project_Team1/Sleep_health_and_lifestyle_dataset.csv"

In [ ]:
def loadCSVFile(csv_path):

  from langchain.document_loaders.csv_loader import CSVLoader

  # Create a CSVLoader instance
  loader = CSVLoader(csv_path)

  # Load the content of the CSV file
  data = loader.load()

  return data[2].page_content

In [ ]:
# printing the content of the file
text = loadCSVFile(csv_path)
print(text)

Person ID: 3
Gender: Male
Age: 28
Occupation: Doctor
Sleep Duration: 6.2
Quality of Sleep: 6
Physical Activity Level: 60
Stress Level: 8
BMI Category: Normal
Blood Pressure: 125/80
Heart Rate: 75
Daily Steps: 10000
Sleep Disorder: None


In [ ]:
import openai
# Initialize the OpenAI API with your API key
openai.api_key = openai_api_key

# You can set up your API key by harcdcoding it here. It is a hacky and bad practice as others will see your secret key clearly and use your account. But, can be used for trying something quick and dirty
# openai.api_key = 'YOUR_OPENAI_API_KEY'

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="""Analyze the "text" data and provide encouraging feedback tailored to the user's progress. Emphasize positive aspects and offer motivational advice for improvement. If the user has sleep disorder or their Stress level is high, please give him/her a comfort words to help them sleep well.

Based on "text" data, generate a brief, encouraging message to help user sleep well.

  """,


  max_tokens=50
)



result_text = response.choices[0].text.strip()
print(result_text)


"Sleep is a vital part of maintaining your health and wellness. Make sure to create a bedtime routine to help relax your mind and body. Try calming techniques like deep breathing, mindfulness meditation, and progressive muscle relaxation. Set yourself up for success by


In [ ]:
from audiocraft.models import MusicGen
from audiocraft.models import MultiBandDiffusion

USE_DIFFUSION_DECODER = False
# Using small model, better results would be obtained with `medium` or `large`.
model = MusicGen.get_pretrained('facebook/musicgen-small')
if USE_DIFFUSION_DECODER:
    mbd = MultiBandDiffusion.get_mbd_musicgen()

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Next, let us configure the generation parameters. Specifically, you can control the following:
* `use_sampling` (bool, optional): use sampling if True, else do argmax decoding. Defaults to True.
* `top_k` (int, optional): top_k used for sampling. Defaults to 250.
* `top_p` (float, optional): top_p used for sampling, when set to 0 top_k is used. Defaults to 0.0.
* `temperature` (float, optional): softmax temperature parameter. Defaults to 1.0.
* `duration` (float, optional): duration of the generated waveform. Defaults to 30.0.
* `cfg_coef` (float, optional): coefficient used for classifier free guidance. Defaults to 3.0.

When left unchanged, MusicGen will revert to its default parameters.

In [ ]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    top_p=0.0,
    temperature=1.0,   # Set standard temperature for variability
    duration=15.0,     # Set the duration of the generated output (e.g., 30 seconds for audio)
    cfg_coef=3.0       # Set classifier-free guidance coefficient
)


Next, we can go ahead and start generating music using one of the following modes:
* Unconditional samples using `model.generate_unconditional`
* Music continuation using `model.generate_continuation`
* Text-conditional samples using `model.generate`
* Melody-conditional samples using `model.generate_with_chroma`

### Text-conditional Generation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from audiocraft.utils.notebook import display_audio

output = model.generate(
    descriptions=[
        #'80s pop track with bassy drums and synth',
        #'90s rock song with loud guitars and heavy drums',
        #'Progressive rock drum and bass solo',
        #'Punk Rock song with loud drum and power guitar',
        #'Bluesy guitar instrumental with soulful licks and a driving rhythm section',
        #'Jazz Funk song with slap bass and powerful saxophone',
        #'drum and bass beat with intense percussions',
        result_text
    ],
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)


***Ignore the following code***


### Melody-conditional Generation

In [ ]:
import torchaudio
from audiocraft.utils.notebook import display_audio

model = MusicGen.get_pretrained('facebook/musicgen-melody')
model.set_generation_params(duration=8)

melody_waveform, sr = torchaudio.load("/content/drive/MyDrive/Colab Notebooks/IS883/ProjectMilestone1/audiocraft-main/assets/bach.mp3")
melody_waveform = melody_waveform.unsqueeze(0).repeat(2, 1, 1)
output = model.generate_with_chroma(
    descriptions=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
    ],
    melody_wavs=melody_waveform,
    melody_sample_rate=sr,
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)